In [5]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
import nltk
import regex
import urllib
import gensim
from gensim.utils import lemmatize
from gensim.utils import simple_preprocess
import spacy 
from spacy.lang.it.stop_words import STOP_WORDS as ita
from wordcloud import WordCloud
from matplotlib import pyplot as plt


In [42]:
!python3 -m spacy download it_core_news_sm

     |████████████████████████████████| 14.5MB 2.6MB/s 
  Created wheel for it-core-news-sm: filename=it_core_news_sm-2.2.5-cp36-none-any.whl size=14471130 sha256=4fab0815f2b85756ce463a5911c764e05845dfbe5ff1df628b4677078c3ad2aa
  Stored in directory: /tmp/pip-ephem-wheel-cache-i3iug6ki/wheels/a1/01/c2/127ab92cc5e3c7f36b5cd4bff28d1c29c313962a2ba913e720
Successfully built it-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('it_core_news_sm')


In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/Research/Datasets/data_processed_ITA_MAY_26.csv')
type(df.Date[0])
for c in range(len(df)):
    li = list(str(df.Date[c]).split(" "))
    for n, i in enumerate(li):
        if i == 'gennaio':
            li[n] = 'January'
        if i == 'febbraio':
            li[n] = 'February'
        if i == 'marzo':
            li[n] = 'March'
        if i == 'aprile':
            li[n] = 'April'
        if i == 'maggio':
            li[n] = 'May'
    df.Date[c] = ", ".join(li)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [38]:
df = pd.read_csv('/content/gdrive/My Drive/Research/Datasets/2010-2012_ITA.csv')
df['Date'] =pd.to_datetime(df.Date)
df =df.dropna().set_index('Date').sort_index()

texts = df.Article.values.tolist()
# df['weekoftheyear'] = [week.weekofyear for week in df.index]
print(len(df))
df.tail()

7807


,Title,Article
Date,,
2012-09-20,Tempi di credit crunch: spiragli in banca per ...,Vocazione verso i mercati esteri e capacità di...
2012-10-03,"Atene nella trappola della recessione: -2,5%il...",La Grecia resterà intrappolata nella recession...
2012-12-02,Infrastutture fra ritardi e carenza di fondi,Il ritardo infrastrutturale e la debolezza ist...
2012-12-06,"Lavoro, parte il confronto",Dalla prossima settimana entra nel vivo il con...
2012-12-21,"Germania, Diw non esclude una recessione nel 2012",La produzione di beni e servizi in Germania ne...


In [39]:
df = df.assign(weekoftheyear=pd.PeriodIndex(df.index, freq='M'))
df

,Title,Article,weekoftheyear
Date,,,
2010-01-03,In Costa d'Avorio riprendono i negoziati diplo...,Si apprestano a raggiungere la Costa d'Avorio ...,2010-01
2010-01-04,Imprenditori stranieri oltre quota 600mila (+9...,La crisi non ha scoraggiato gli imprenditori s...,2010-01
2010-01-04,Nel 2010 seimila aziende in cassa integrazione,2010: l'anno delle 6mila aziende in cassa inte...,2010-01
2010-01-04,Piazza Affari chiude in rialzo con Eni e Fiat ...,Le Borse europee (se si esclude il listino ted...,2010-01
2010-01-05,Il referendum sull'accordo per Mirafiori si te...,Conto alla rovescia per la consultazione dei l...,2010-01
...,...,...,...
2012-09-20,Tempi di credit crunch: spiragli in banca per ...,Vocazione verso i mercati esteri e capacità di...,2012-09
2012-10-03,"Atene nella trappola della recessione: -2,5%il...",La Grecia resterà intrappolata nella recession...,2012-10
2012-12-02,Infrastutture fra ritardi e carenza di fondi,Il ritardo infrastrutturale e la debolezza ist...,2012-12


In [40]:
#regular expression: formatting text
def formatting(texts):
    texts = [regex.sub('\s+', ' ', text) for text in texts] #remove new lines
    texts = [regex.sub("\"", "", text) for text in texts] #remove quotations
    texts = [regex.sub("\'", "", text) for text in texts] #remove quotations
    return texts
formatted_text = formatting(texts)

In [43]:

import it_core_news_sm
stop_words = list(ita)

url = "https://countwordsfree.com/stopwords/italian/txt"
file = urllib.request.urlopen(url)
stop_ext = [line.decode("utf-8").strip() for line in file]
stop_words = stop_words + stop_ext
stop_words.extend(['q','w','e','r','t','y','u','i','o','p','a','s','d','f','g','h','j','k','l','z','x','c','v','b','n','m'])
lemma = it_core_news_sm.load()

def token(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True)) 

def bigram_trigram(texts):
    bigram = gensim.models.Phrases(texts)
    trigram = gensim.models.Phrases(bigram[texts])
    bigram_ = gensim.models.phrases.Phraser(bigram)
    trigram_ = gensim.models.phrases.Phraser(trigram)
    texts = [[word for word in doc if word not in stop_words] for doc in texts] 
    bi = [bigram_[text] for text in texts]
    tri = [trigram_[bigram_[b]] for b in bi]
    return tri
def processing(texts, stop_words=stop_words):
    texts = [[word for word in doc if word not in stop_words] for doc in texts]
    output = []
    for text in texts:
        doc = lemma(" ".join(text)) 
        output.append([token.lemma_ for token in doc])
    # remove stopwords once more after lemmatization
    output = [[word for word in doc if word not in stop_words] for doc in output]
    return output

tokens = list(token(formatted_text))
bi_tri = bigram_trigram(tokens)
final_data = processing(bi_tri)
df['ProcessedArticles'] = final_data
df

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


,Title,Article,weekoftheyear,ProcessedArticles
Date,,,,
2010-01-03,In Costa d'Avorio riprendono i negoziati diplo...,Si apprestano a raggiungere la Costa d'Avorio ...,2010-01,"[apprestare, raggiungere, costare, davorio, en..."
2010-01-04,Imprenditori stranieri oltre quota 600mila (+9...,La crisi non ha scoraggiato gli imprenditori s...,2010-01,"[crisi, scoraggiare, imprenditori_stranieri, a..."
2010-01-04,Nel 2010 seimila aziende in cassa integrazione,2010: l'anno delle 6mila aziende in cassa inte...,2010-01,"[lanno, azienda, cassa_integrazione_straordina..."
2010-01-04,Piazza Affari chiude in rialzo con Eni e Fiat ...,Le Borse europee (se si esclude il listino ted...,2010-01,"[borse_europee, escludere, listare, tedesco, a..."
2010-01-05,Il referendum sull'accordo per Mirafiori si te...,Conto alla rovescia per la consultazione dei l...,2010-01,"[contare, rovescio, consultazione, lavoratore,..."
...,...,...,...,...
2012-09-20,Tempi di credit crunch: spiragli in banca per ...,Vocazione verso i mercati esteri e capacità di...,2012-09,"[vocazione, mercati_esteri, capacitare, aggreg..."
2012-10-03,"Atene nella trappola della recessione: -2,5%il...",La Grecia resterà intrappolata nella recession...,2012-10,"[grecia, restera, intrappolare, recessione, pr..."
2012-12-02,Infrastutture fra ritardi e carenza di fondi,Il ritardo infrastrutturale e la debolezza ist...,2012-12,"[ritardare, debolezza, istituzionale, farvi, f..."


In [44]:
def search_risk(text,n):
    word = r'\W*([\w]+)'
    rischio = regex.findall(r'{}\W*{}{}'.format(word*n,'rischio',word*n), text)
    rischiare = regex.findall(r'{}\W*{}{}'.format(word*n,'rischiare',word*n), text)
    incertezza = regex.findall(r'{}\W*{}{}'.format(word*n,'incertezza',word*n), text)
    paura = regex.findall(r'{}\W*{}{}'.format(word*n,'paura',word*n), text)
    timore = regex.findall(r'{}\W*{}{}'.format(word*n,'timore',word*n), text)
    return rischio+rischiare+incertezza+paura+timore


In [45]:
risk = [(search_risk(' '.join(data),10)) for data in final_data ]

In [45]:
df['Risk'] = risk


In [46]:
stop_words = list(ita)

url = "https://countwordsfree.com/stopwords/italian/txt"
file = urllib.request.urlopen(url)
stop_ext = [line.decode("utf-8").strip() for line in file]
stop_words = stop_words + stop_ext
stop_words.extend(['q','w','e','r','t','y','u','i','o','p','a','s','d','f','g','h','j','k','l','z','x','c','v','b','n','m'])
stop_words.extend(['calere','indicatori_re','gennaio','febbraio','marzo','aprile','maggio','giugno','luglio','agosto','settembre','ottobre','novembre','dicembre','ce','irlanda','italia','litalia','cavallo',])

def frequency(data,top_N,term):
  freq = []
  for i in df['weekoftheyear'].unique():
    week_data = df[(df['weekoftheyear'] == i)][term]
    flat_list = [item for sublist in week_data for item in sublist]
    # flat_list1 = [item for sublist in week_data for item in sublist]

    flat_list1 = [item for sublist in flat_list for item in sublist]
    output = [word for word in flat_list1 if word not in stop_words]
    word_dist = nltk.FreqDist(output)
    temp = pd.DataFrame(word_dist.most_common(),
                      columns=['Word', 'Frequency'])
    temp['Percentage'] = temp.Frequency / len(temp.Word) * 100
    freq.append(temp.head(20))
  return freq


In [47]:
risk = frequency(df['Risk'],20,'Risk')

KeyError: ignored

In [ ]:
risk[0]

In [ ]:
risk

In [ ]:
df_risk = []
ind = list(df.weekoftheyear.unique())
for i in range(len(risk)):
  a = (risk[i].Word.head(5).to_list())
  df_risk.append(a)
df_risk = pd.DataFrame(df_risk,columns=['Word1','Word2','Word3','Word4','Word5'])
# df_risk.to_csv('/content/gdrive/My Drive/Research/Datasets/risk_ita2010.csv')
df_risk.set_index(ind)


In [170]:
ind = list(df.weekoftheyear.unique())

In [ ]:
df1 = pd.read_csv('/content/gdrive/My Drive/Research/final_risk_10_ITA.csv')
df1.head()

,Date,Article,Title,weekoftheyear,ProcessedArticles,Risk
0,2020-01-19,Aumenta la preoccupazione per l’ondata di polm...,Timori per il virus polmonare cinese: i casi p...,3,"['aumentare', 'preoccupazione', 'ondata', 'pol...","[('polmonite', 'cina', 'causare', 'ceppo', 'co..."
1,2020-01-21,NEW YORK - Gli Stati Uniti riportano il primo ...,"Virus cinese, primo caso negli Usa. Riunione d...",4,"['new_york', 'unito', 'riportare', 'coronaviru...","[('cdc', 'riferire', 'mass', 'medio', 'persona..."
2,2020-01-21,I timori per il contagio del coronavirus in Ci...,"Europa limita perdite sul finale, moda e lusso...",4,"['timore', 'contagiare', 'coronavirus', 'cina'...","[('sulleconomia', 'atteso', 'consentire', 'fra..."
3,2020-01-21,"Vendite sui titoli del lusso, che risentono de...",Il lusso sotto pressione in Borsa per il virus...,4,"['vendita', 'lussare', 'risentire', 'timore', ...","[('europa', 'kering', 'parigi', 'cattivo', 'ca..."
4,2020-01-22,"L’Oms ha ritenuto opportuno, per ora, non dich...",Virus cinese: sembra escluso il caso del conta...,4,"['oms', 'ritenere', 'opportuno', 'dichiarare',...","[('medico', 'loms', 'chiedere', 'cina', 'conti..."


In [ ]:
df_risk

,Word1,Word2,Word3,Word4,Word5
2009-12-28/2010-01-03,None,None,None,None,None
2010-01-04/2010-01-10,azienda,referendum,confino,crisi,mettere
2010-01-11/2010-01-17,None,None,None,None,None
2010-01-18/2010-01-24,None,None,None,None,None
2010-02-01/2010-02-07,None,None,None,None,None
...,...,...,...,...,...
2012-09-17/2012-09-23,None,None,None,None,None
2012-10-01/2012-10-07,None,None,None,None,None
2012-11-26/2012-12-02,None,None,None,None,None
2012-12-03/2012-12-09,None,None,None,None,None


In [ ]:
def f(x):
     return pd.Series(dict(A = x['A'].sum(), 
                        B = x['B'].sum(), 
                        C = "{%s}" % ', '.join(x['C'])))

In [ ]:
a = df.groupby(pd.Grouper(freq='M'))['ProcessedArticles'].sum()

In [ ]:
df = pd.DataFrame(a)

In [ ]:
final_data

In [119]:
a = pd.merge(risk[14],risk[15],how = 'right', on='Word')
risk[11]['change'] = (a['Frequency_y']-a['Frequency_x'])/(a[['Frequency_y','Frequency_x']]).values.max(1)
risk[11].dropna().sort_values('change',ascending=False)

,Word,Frequency,Percentage,change
9,produrre,78,0.858369,0.490566
6,crisi,93,1.023440,0.333333
8,euro,90,0.990426,0.270270
5,potere,99,1.089468,0.108108
7,presidente,93,1.023440,0.068966
4,politico,100,1.100473,-0.102564
2,mercato,121,1.331573,-0.166667
1,dovere,128,1.408606,-0.183333
3,aumentare,108,1.188511,-0.230769
0,crescita,150,1.650710,-0.278689


In [121]:
risk[15]

,Word,Frequency,Percentage
0,euro,53,0.945247
1,crescita,49,0.873908
2,aumentare,45,0.802568
3,presidente,44,0.784733
4,impresa,40,0.713394
5,crescere,37,0.659889
6,potere,37,0.659889
7,mese,37,0.659889
8,dato,36,0.642055
9,sviluppare,35,0.624220


In [123]:
df

,Title,Article,weekoftheyear,ProcessedArticles,Risk
Date,,,,,
2010-01-03,In Costa d'Avorio riprendono i negoziati diplo...,Si apprestano a raggiungere la Costa d'Avorio ...,2010-01,"[apprestare, raggiungere, costare, davorio, en...",[]
2010-01-04,Imprenditori stranieri oltre quota 600mila (+9...,La crisi non ha scoraggiato gli imprenditori s...,2010-01,"[crisi, scoraggiare, imprenditori_stranieri, a...",[]
2010-01-04,Nel 2010 seimila aziende in cassa integrazione,2010: l'anno delle 6mila aziende in cassa inte...,2010-01,"[lanno, azienda, cassa_integrazione_straordina...","[(storico, cassare, ordinario, straordinario, ..."
2010-01-04,Piazza Affari chiude in rialzo con Eni e Fiat ...,Le Borse europee (se si esclude il listino ted...,2010-01,"[borse_europee, escludere, listare, tedesco, a...","[(guadagnare, singoli_titoli, scattare, fiat_s..."
2010-01-05,Il referendum sull'accordo per Mirafiori si te...,Conto alla rovescia per la consultazione dei l...,2010-01,"[contare, rovescio, consultazione, lavoratore,...","[(landini, ribadire, firmare, tecnico, sullacc..."
...,...,...,...,...,...
2012-09-20,Tempi di credit crunch: spiragli in banca per ...,Vocazione verso i mercati esteri e capacità di...,2012-09,"[vocazione, mercati_esteri, capacitare, aggreg...",[]
2012-10-03,"Atene nella trappola della recessione: -2,5%il...",La Grecia resterà intrappolata nella recession...,2012-10,"[grecia, restera, intrappolare, recessione, pr...",[]
2012-12-02,Infrastutture fra ritardi e carenza di fondi,Il ritardo infrastrutturale e la debolezza ist...,2012-12,"[ritardare, debolezza, istituzionale, farvi, f...",[]
